## knock 052 数値データを二値(0と1)データに変換する

In [4]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列が省略されないようにする

polars.config.Config

### データを読み込む

In [5]:
df_receipt = pl.read_csv("../docker/work/data/receipt.csv")
display(df_receipt.head())

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001…","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097…","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014…","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000…","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050…","""P060102007""",1,90


### ノック

In [6]:
(
    df_receipt
    # 非会員は除く
    .filter( ~pl.col("customer_id").str.starts_with("Z") )
    # 顧客IDごとの合計売上を集計する
    .group_by(by = "customer_id")
    .agg( pl.sum("amount").alias("sum_amount") )
    # 売上が2000円以上か否かの二値化列を追加する
    .with_columns(
        pl.col("sum_amount").map_elements(lambda x: 1 if x > 2000 else 0).alias("salse_flg")
    )
    .head(n = 10)
    # 書籍のIDを抽出する
    
)

customer_id,sum_amount,salse_flg
str,i64,i64
"""CS013514000050…",2633,1
"""CS014411000019…",1698,0
"""CS019315000110…",1450,0
"""CS003415000065…",2291,1
"""CS016411000023…",1271,0
"""CS022513000057…",318,0
"""CS005512000195…",563,0
"""CS005214000028…",1094,0
"""CS021803000003…",1180,0
